# Este notebook é voltado para testes de implantação de um modelo que classifica os generos de uma música utilizando somente sua letra

neste documento você encontrara todos os passos básicos de processamento de dados e implementação de um modelo básico

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn import utils
#para os testes com Doc2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from tqdm import tqdm

# Carregamento dos dados
### Nesta primeira parte, é utilizada uma função que carrega todos os csvs por gêneros, e concatenam eles para o mesmo DataSet, adicionando a coluna gênero, já em formato numérico

In [1]:
def concat_csvs(all_files):
    df = {}
    i = 0
    for f in all_files:
        df[i] = pd.read_csv(f)
        df[i]['genero'] = i + 1
        i+= 1
    all_pds = pd.concat(df, ignore_index=True)
    return all_pds

In [5]:
import glob
from pathlib import Path

path = 'C:\\Users\\davia\\Documents\\Projetos\\Finch\\Data\\generos\\'
all_files = glob.glob(path + "\*.csv")



df_from_each_file = concat_csvs(all_files)

C:\Users\davia\Documents\Projetos\Finch\Data\generos\bossa_nova.csv
C:\Users\davia\Documents\Projetos\Finch\Data\generos\funk.csv
C:\Users\davia\Documents\Projetos\Finch\Data\generos\gospel.csv
C:\Users\davia\Documents\Projetos\Finch\Data\generos\sertanejo.csv


# Aqui começam as funções de limpeza dos dados
- São retirados caracteres especiais
- espaçamentos duplos são retirados para não criar problemas no split
-  o texto é convertido para caixa baixa
- são retiradas pontuações
- são deletadas as colunas duplicadas

In [5]:
def clean_text(text):
    text = text.replace('\n',' ')
    text = text.replace('  ',' ')
    text = text.lower()
    text = text.replace(',','')
    text = text.replace('?','')
    text = text.replace('!','')
    text = text.replace('.','')
    return text

In [6]:
df_from_each_file['lyric'] = df_from_each_file['lyric'].apply(clean_text)

In [7]:
df_from_each_file.drop_duplicates(inplace = True)
df_from_each_file = df_from_each_file.sample(frac=1)

In [8]:
df_from_each_file.to_csv('Data\\final.csv', encoding='utf-8')

In [9]:
y = []
text = []

for index, row in df_from_each_file.iterrows():
    y.append(row[1])
    text.append(row[0])

## Aqui utilizamos a funcção TfidfVectorizer do sklearn, com o parametro stop_words para portugues, e tambem, os valores gerados pela função são normalizados e escalados, concluindo a limpeza e transformação dos dados
## Tambem é feito o Split dos dados

In [10]:
countVec = TfidfVectorizer(stop_words = stopwords.words('portuguese'), sublinear_tf=True)

In [11]:
x = countVec.fit_transform(text)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
lr = LogisticRegression(max_iter= 1000)
lr.fit(x_train,y_train)

In [ ]:
y_pred = lr.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
print("acurácia da regressão lógica: %s"%(accuracy))

In [ ]:
np.mean(cross_val_score(lr, x, y, scoring='accuracy'))

# Aqui começa o teste do modelo de RandomForest

## Foi utilizado o método de acurácia

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

In [ ]:
y_pred1 = rf.predict(x_test)
accuracy = accuracy_score(y_test,y_pred1)

print("acurácia da Random Forest: %s"%(accuracy))

# Em seguida, o modelo de treino do Doc2Vec

In [ ]:
genTrain, genTest = train_test_split(df_from_each_file, test_size=0.3, random_state=42)

In [ ]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

train_tagged = genTrain.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['lyric']), tags=[r.genero]), axis=1)
test_tagged = genTest.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['lyric']), tags=[r.genero]), axis=1)

In [ ]:
cores = multiprocessing.cpu_count()

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

# Modelo de regressão logistica para utilização do Doc2Vec

In [ ]:
y_train1, X_train1 = vec_for_learning(model_dbow, train_tagged)
y_test1, X_test1 = vec_for_learning(model_dbow, test_tagged)

logreg = LogisticRegression()
logreg.fit(X_train1, y_train1)
y_pred1 = logreg.predict(X_test1)

from sklearn.metrics import accuracy_score, f1_score

print('acurácia da regressão lógica %s' % accuracy_score(y_test1, y_pred1))

# Modelo de Random Forest para utilização do Doc2Vec

In [ ]:
randforest = RandomForestClassifier()
randforest.fit(X_train1,y_train1)

In [ ]:
y_pred1 = randforest.predict(X_test1)
accuracy = accuracy_score(y_test1,y_pred1)

print("acurácia da Random Forest com Doc2Vec: %s"%(accuracy))